In [ ]:
!pip install pyresparser
!pip install nltk
import nltk
nltk.download('stopwords')
import os
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import filedialog
import tkinter.font as font
from functools import partial
from pyresparser import ResumeParser
from sklearn import datasets, linear_model
#... (rest of the code remains the same)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=8145eb418e3e93c04ef25e4f0863734063eb4ed05e125213780a90c41e740bb4
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:


# Import necessary libraries
import os
import pandas as pd
import numpy as np
import gradio as gr
from pyresparser import ResumeParser
from sklearn import datasets, linear_model

# Define the training class for the model
class TrainModel:

    def __init__(self):
        self.mul_lr = None

    def train(self):
        data = pd.read_csv('/content/training_dataset.csv')
        array = data.values

        # Encode gender as numerical values
        for i in range(len(array)):
            array[i][0] = 1 if array[i][0] == "Male" else 0

        df = pd.DataFrame(array)
        maindf = df[[0, 1, 2, 3, 4, 5, 6]]
        mainarray = maindf.values
        train_y = df[7].values

        self.mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=1000)
        self.mul_lr.fit(mainarray, train_y)

    def test(self, test_data):
        try:
            test_predict = list(map(int, test_data))
            y_pred = self.mul_lr.predict([test_predict])
            return y_pred[0]  # Return the predicted personality
        except Exception as e:
            return str(e)

# Function to parse resume using pyresparser
def parse_resume(cv_path):
    try:
        data = ResumeParser(cv_path).get_extracted_data()
        # Clean data
        try:
            del data['name']
            if len(data.get('mobile_number', '')) < 10:
                del data['mobile_number']
        except KeyError as e:
            print(f"KeyError: {e}")
            pass
        return data
    except Exception as e:
        print(f"Error parsing resume: {e}")
        return {}

# Function to predict personality based on inputs
def predict_personality(applicant_name, age, gender, cv_path, openness, neuroticism, conscientiousness, agreeableness, extraversion):
    try:
        # Convert gender to numerical value
        gender = 1 if gender == "Male" else 0
        # Create personality values based on input
        personality_values = [gender, age, int(openness), int(neuroticism), int(conscientiousness), int(agreeableness), int(extraversion)]

        print(f"Personality values: {personality_values}")

        personality = model.test(personality_values)
        resume_data = parse_resume(cv_path.name)

        result = {
            "Applicant Name": applicant_name,
            "Age": age,
            "Predicted Personality": personality,
            "Resume Data": resume_data
        }

        return result
    except Exception as e:
        return {"error": str(e)}

# Initialize and train the model
model = TrainModel()
model.train()

# Define the Gradio interface
inputs = [
    gr.Textbox(label="Applicant Name"),
    gr.Number(label="Age"),
    gr.Radio(label="Gender", choices=["Male", "Female"]),
    gr.File(label="Upload Resume"),
    gr.Slider(label="Openness (1-10)", minimum=1, maximum=10, step=1),
    gr.Slider(label="Neuroticism (1-10)", minimum=1, maximum=10, step=1),
    gr.Slider(label="Conscientiousness (1-10)", minimum=1, maximum=10, step=1),
    gr.Slider(label="Agreeableness (1-10)", minimum=1, maximum=10, step=1),
    gr.Slider(label="Extraversion (1-10)", minimum=1, maximum=10, step=1)
]

outputs = gr.JSON(label="Prediction Result")

# Launch the Gradio app
gr.Interface(fn=predict_personality, inputs=inputs, outputs=outputs, title="Personality Prediction System").launch()


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36a47a2573129dfb79.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
